# Logistic Regression Example

In this tutorial, we'll walk through how to use our automatic differentiation engine to create and train a logistic regression model to predict heart disease. 

The data can be found [here](https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset).

Our goal here is not to create the best model to fit our data, but to simply show how to use our automatic differentiation engine to create, train, and test a logistic regression model. Thus, our model will be very simplistic for pedagogical purposes.

# Data Preprocessing

We'll take look at our data and then preprocess it to be in a form appropriate to use with our automatic differentiation engine.

Feel free to skip this section as it doesn't directly relate to how to use our automatic differentiation engine.

Let's first import some necessary libraries.

In [2]:
from autograd import Variable, LogisticRegressionLayer
import autograd
from tqdm import tqdm
import pandas as pd
import numpy as np
import random

Let's load the data. 

In [3]:
csv_path = './data/framingham.csv'
df = pd.read_csv(csv_path)
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


Let's clean our data.

Since this tutorial is focused on using our automatic differentiaion engine rather than on machine learning itself, we'll avoid doing a deep dive into the data analysis rationalizing the methods we here to clean our data. More examples of analysis on this dataset can be found [here](https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset/notebooks).

Let's start by dropping any null values

In [4]:
df.dropna(axis=0, inplace=True)
assert df.isnull().sum().sum() == 0

Let's normalize our data.

In [5]:
for column in df.columns:
    if column != 'TenYearCHD':
        df[column] = (df[column] - df[column].mean()) / df[column].std()

df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1.119602,-1.232411,1.974939,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,-0.948978,-1.193947,-1.078733,0.292010,0.356321,-0.203016,0
1,-0.892931,-0.414848,0.019247,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,0.298253,-0.514811,-0.160096,0.724911,1.608249,-0.244850,0
2,1.119602,-0.181259,-0.958599,1.021975,0.920563,-0.176877,-0.075976,-0.672768,-0.166761,0.184868,-0.220518,-0.243609,-0.108914,-0.060988,-0.495852,0
3,-0.892931,1.337073,0.997093,1.021975,1.759377,-0.176877,-0.075976,1.485991,-0.166761,-0.268670,0.798187,1.009079,0.688016,-0.895606,0.884661,1
4,-0.892931,-0.414848,0.997093,1.021975,1.172207,-0.176877,-0.075976,-0.672768,-0.166761,1.091946,-0.107329,0.090441,-0.659878,0.773630,0.131654,0


Our model will be very simple. We'll create a logistic regression model that takes in a value from each column and will return whether or not the patient has a 10 year risk of heart disease.

Our automatic differentiation engine expects numpy arrays. 

Let's convert the data into two numpy arrays, one for the inputs and one for the outputs.

In [6]:
input_df = df[['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
output_df = df['TenYearCHD']
input_matrix = input_df.values
output_vector = output_df.values

In [7]:
type(input_matrix)

numpy.ndarray

In [8]:
input_matrix.shape

(3658, 15)

In [9]:
type(output_vector)

numpy.ndarray

In [10]:
output_vector.shape

(3658,)

Let's now split our data into a training set and test set. We'll use 70% of our data for training and 30% for testing. We'll manually set the random seeds for reproducibility purposes.

In [11]:
random.seed(0)
np.random.seed(0)

row_count = input_matrix.shape[0]
row_indices = list(range(row_count))
random.shuffle(row_indices)

last_training_row_position = round(row_count*0.75)
training_indices = row_indices[:last_training_row_position+1]
testing_indices = row_indices[last_training_row_position+1:]

training_input_matrix = input_matrix[training_indices]
training_output_vector = output_vector[training_indices]
testing_input_matrix = input_matrix[testing_indices]
testing_output_vector = output_vector[testing_indices]

A logistic regression model can be implemented as a dense linear layer with a sigmoid activation. 

Our automatic differentiation engine provides a layer that does exactly that.

In [12]:
logistic_regression_layer = LogisticRegressionLayer(15, 1)

First, we'll need to establish some hyperparameters for training our linear regression model. We'll be using stochastic gradient descent.

In practice, we might use various batch sizes or shuffle our training data between epochs, but we'll forgo this for the sake of simplicity as the purposes of this tutorial is to show how to use our automatic differentiation engine rather than finding the best model possible.

In [13]:
number_of_epochs = 40
learning_rate = 1e-3
sgd = autograd.optimizer.SGD(learning_rate)

Let's write our training loop. We'll use cross entropy as the loss function along with L2 regularization.

In [14]:
epoch_iterator = tqdm(range(number_of_epochs))
for epoch_index in epoch_iterator:
    mean_loss = 0 
    for x, y in zip(training_input_matrix, training_output_vector):
        y_hat = logistic_regression_layer(x)
        loss = y_hat.bce_loss(y)
        mean_loss += loss.data.item()
        sgd.take_training_step(loss)
    mean_loss /= len(training_output_vector)
    epoch_iterator.set_description(f'Mean Loss: {mean_loss:.8f}')
    epoch_iterator.refresh()

Mean Loss: 0.37751710: 100%|██████████| 40/40 [00:30<00:00,  1.32it/s]


Let's now see how our model performs on our test set.

In [15]:
true_positive_count = 0
true_negative_count = 0
false_positive_count = 0
false_negative_count = 0

for x, y in zip(testing_input_matrix, testing_output_vector):
    y_hat = logistic_regression_layer(x)
    y_hat = y_hat.round()
    is_correct = y_hat == y
    is_positive = y == 1
    if is_correct:
        if is_positive:
            true_positive_count += 1
        else:
            true_negative_count += 1
    else:
        if is_positive:
            false_positive_count += 1
        else:
            false_negative_count += 1

precision = true_positive_count/(true_positive_count+false_positive_count)
recall = true_positive_count/(true_positive_count+false_negative_count)
f1 = 2 * precision * recall / (precision + recall)

accuracy = (true_positive_count + true_negative_count) / (true_positive_count + true_negative_count + false_positive_count + false_negative_count)

print(f'True Positives:  {true_positive_count}')
print(f'True Negatives:  {true_negative_count}')
print(f'False Positives: {false_positive_count}')
print(f'False Negatives: {false_negative_count}')
print()
print(f'Precision: {precision}')
print(f'Recall:    {recall}')
print(f'F1:        {f1}')
print(f'Accuracy:  {accuracy}')

True Positives:  13
True Negatives:  766
False Positives: 124
False Negatives: 10

Precision: 0.0948905109489051
Recall:    0.5652173913043478
F1:        0.16249999999999998
Accuracy:  0.8532311062431545
